# Volunteer model loss calculation

This notebook takes a `subject_id` and calculates the MSE of each volunteer's model to the galaxy data. These losses are then glued to the notebook for use elsewhere.

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from gzbuilder_analysis import parsing, rendering, fitting
import lib.galaxy_utilities as gu
import scrapbook as sb

/opt/apps/pkgs/anaconda3/2019.03/intel64/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


In [3]:
subject_id = 20902040

In [4]:
# Parameters
subject_id = 21096782


In [5]:
diff_data_df = pd.read_pickle('lib/fitting_metadata.pkl')

In [6]:
def get_losses_by_user(subject_id):
  diff_data = diff_data_df.loc[subject_id]
  psf = diff_data['psf']
  pixel_mask = np.array(diff_data['pixel_mask'])[::-1]
  galaxy_data = np.array(diff_data['galaxy_data'])[::-1]
  image_size = galaxy_data.shape[0]
  size_diff = diff_data['size_diff']
  
  cls = gu.classifications.query('subject_ids == {}'.format(subject_id))
  names = cls.user_name 
  annotations = cls['annotations'].apply(json.loads)

  models = annotations.apply(
    parsing.parse_annotation,
    size_diff=size_diff
  )

  tqdm.pandas(
    desc='Rendering models'.format(subject_id),
    leave=False
  )
  rendered = models.progress_apply(
    rendering.calculate_model,
    image_size=image_size,
    psf=psf
  ).rename('rendered')

  tqdm.pandas(
    desc='Calculating differences'.format(subject_id),
    leave=False
  )
  losses = rendered.apply(
    fitting.loss,
    args=(galaxy_data, pixel_mask)
  ).rename('loss')
  all_losses = pd.Series(
    losses.values,
    index=names.values
  ).rename(subject_id)
  best_losses = pd.concat((
    all_losses.where(
      ~all_losses.duplicated(keep=False)
    ).dropna(), # all non-duplicates
    all_losses.where(
      all_losses.duplicated(keep=False)
    ).dropna().groupby(level=0).min() # best value of duplicates
  ), axis=0)
  return best_losses

In [7]:
losses = get_losses_by_user(subject_id)
losses

Rendering models:   0%|          | 0/30 [00:00<?, ?it/s]

Rendering models:   7%|▋         | 2/30 [00:07<01:45,  3.78s/it]

Rendering models:  10%|█         | 3/30 [00:07<01:14,  2.75s/it]

Rendering models:  13%|█▎        | 4/30 [00:08<00:54,  2.10s/it]

Rendering models:  17%|█▋        | 5/30 [00:08<00:40,  1.60s/it]

...

Rendering models:  90%|█████████ | 27/30 [00:18<00:01,  2.09it/s]

Rendering models:  93%|█████████▎| 28/30 [00:18<00:00,  2.08it/s]

Rendering models:  97%|█████████▋| 29/30 [00:18<00:00,  2.56it/s]

Rendering models: 100%|██████████| 30/30 [00:19<00:00,  2.01it/s]

not-logged-in-3b0b929f4475fd276561    0.004766
Linda_J._Berkel                       0.000243
Jnursssmith5263                       0.001061
not-logged-in-c3e635069550aa98e5fd    0.003371
nurenai                               0.000289
Jonas_Cross                           0.000371
jnarayanbvg                           0.006261
not-logged-in-c6917d9b19f41e6c2995    0.000482
not-logged-in-c6917d9b19f41e6c2995    0.001460
awright5                              0.002416
cypherk78                             0.000934
Maverna                               0.000558
AlexYoung35                           0.000448
clairedeu                             0.000469
Planetme                              0.000838
not-logged-in-a71e7261972631065ed8    0.000320
rhondacard                            0.003316
not-logged-in-fcfcbc88feb1aa0ed0a7    0.001072
Lavadude                              0.001986
equidad1                              0.000238
ktayl47                               0.000368
not-logged-in

In [8]:
sb.glue('subject_id', subject_id)
sb.glue('losses', losses.to_dict())